In [1]:
import pandas as pd
import re 

https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016

# 1 скачать и загрузить данные

In [2]:
data = pd.read_csv("/home/olga/Documents/teaching/machine_Learning/kaggle/suisade/master.csv")

# 2  посмотреть общую информацию о данных

In [21]:
data.head()

,country,year,sex,age,suicides_no,population,suicides/100k_pop,HDI_for_year,gdp_for_year($),gdp_per_capita($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,NaN,2156624900,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,NaN,2156624900,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,NaN,2156624900,796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,NaN,2156624900,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,NaN,2156624900,796,Boomers


In [4]:
data.tail()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,Uzbekistan2014,0.675,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z
27819,Uzbekistan,2014,female,55-74 years,21,1438935,1.46,Uzbekistan2014,0.675,"63,067,077,179",2309,Boomers


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 11 columns):
country              27820 non-null object
year                 27820 non-null int64
sex                  27820 non-null object
age                  27820 non-null object
suicides_no          27820 non-null int64
population           27820 non-null int64
suicides/100k_pop    27820 non-null float64
HDI_for_year         8364 non-null float64
gdp_for_year($)      27820 non-null int64
gdp_per_capita($)    27820 non-null int64
generation           27820 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 2.3+ MB


In [19]:
data.describe()

,year,suicides_no,population,suicides/100k_pop,HDI_for_year,gdp_for_year($),gdp_per_capita($)
count,27820.000000,27820.000000,2.782000e+04,27820.000000,8364.000000,2.782000e+04,27820.000000
mean,2001.258375,242.574407,1.844794e+06,12.816097,0.776601,4.455810e+11,16866.464414
std,8.469055,902.047917,3.911779e+06,18.961511,0.093367,1.453610e+12,18887.576472
min,1985.000000,0.000000,2.780000e+02,0.000000,0.483000,4.691962e+07,251.000000
25%,1995.000000,3.000000,9.749850e+04,0.920000,0.713000,8.985353e+09,3447.000000
50%,2002.000000,25.000000,4.301500e+05,5.990000,0.779000,4.811469e+10,9372.000000
75%,2008.000000,131.000000,1.486143e+06,16.620000,0.855000,2.602024e+11,24874.000000
max,2016.000000,22338.000000,4.380521e+07,224.970000,0.944000,1.812071e+13,126352.000000


In [20]:
data.describe(include=[object])

,country,sex,age,generation
count,27820,27820,27820,27820
unique,101,2,6,6
top,Iceland,male,35-54 years,Generation X
freq,382,13910,4642,6408


In [8]:
data.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype='object')

In [9]:
data.columns = ['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k_pop', 'country-year', 'HDI_for_year',
       'gdp_for_year($)', 'gdp_per_capita($)', 'generation']

# 3 перевести данные в более читаемый формат

### 3.1  перевести gdp_for_year в числа из object 

(если не понимаешь, почему не получается работать с колонкой - подсказка 1 в файле)

In [10]:
def str_gdp2num(num:str)->int:
    num = num.split(",")
    num = "".join(num)
    return int(num)

str_gdp2num("2,156,624,900")

2156624900

In [11]:
data["gdp_for_year($)"] = data["gdp_for_year($)"].apply(str_gdp2num)
data["gdp_for_year($)"].head()

0    2156624900
1    2156624900
2    2156624900
3    2156624900
4    2156624900
Name: gdp_for_year($), dtype: int64

### 3.2  проверить, что country-year дублирует информцию. если да - удалить её

один из возможных вариантов разделения текста и цифр - модуль re и функция groups тебе в помощь

если не получается - подсказка 2

In [12]:
def split_country_year(data:str)->list:
    temp = re.compile("([a-zA-Z\ ]+)([0-9]+)")
    res = temp.match(data).groups() 
    return res
    
split_country_year("djsd kjfk87435")

('djsd kjfk', '87435')

In [13]:
data["country-year"] = data["country-year"].apply(split_country_year)

In [14]:
(data.country == data["country-year"].apply(lambda x:x[0])).all()

True

In [15]:
(data.year == data["country-year"].apply(lambda x:int(x[1]))).all()

True

In [17]:
data.drop(["country-year"], axis=1, inplace=True)
data

,country,year,sex,age,suicides_no,population,suicides/100k_pop,HDI_for_year,gdp_for_year($),gdp_per_capita($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,NaN,2156624900,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,NaN,2156624900,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,NaN,2156624900,796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,NaN,2156624900,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,NaN,2156624900,796,Boomers
5,Albania,1987,female,75+ years,1,35600,2.81,NaN,2156624900,796,G.I. Generation
6,Albania,1987,female,35-54 years,6,278800,2.15,NaN,2156624900,796,Silent
7,Albania,1987,female,25-34 years,4,257200,1.56,NaN,2156624900,796,Boomers
8,Albania,1987,male,55-74 years,1,137500,0.73,NaN,2156624900,796,G.I. Generation
9,Albania,1987,female,5-14 years,0,311000,0.00,NaN,2156624900,796,Generation X


### 3.3 проверить есть ли связь между возрастом и поколением

если информация дублируется - удалить

In [31]:
age_gen.age.value_counts()

35-54 years    4642
75+ years      4642
15-24 years    4642
25-34 years    4642
55-74 years    4642
5-14 years     4610
Name: age, dtype: int64

In [36]:
age_gen.generation.value_counts()

Generation X       6408
Silent             6364
Millenials         5844
Boomers            4990
G.I. Generation    2744
Generation Z       1470
Name: generation, dtype: int64

In [24]:
age_gen = data[["age", "generation"]]

In [41]:
age_gen[age_gen.age == "75+ years"].generation.value_counts()

Silent             2528
G.I. Generation    2114
Name: generation, dtype: int64

### 3.4 проверить, уникален ли год для страны. если нет - какие года есть

In [53]:
country_info = pd.DataFrame
for country in set(data.country.values):
    print(country)
    print(data[data.country == country].year.unique())

Luxembourg
[1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015]
Iceland
[1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015 2016]
Armenia
[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016]
Russian Federation
[1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015]
Kiribati
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001]
Brazil
[1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015]
Oman
[2009 2010 2014]
Chile
[1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 200

# 4 анализ

### 4.1 посчтать количество самоубийств по странам  годам

то есть процент и количество человек умерло в стране независимо от возраста, пола, ...

можешь где-то хранить (листы, словари, датафрейм), можешь просто распечатать

In [56]:
country_info = pd.DataFrame
for country in data.country.unique():
    data_country = data[data.country == country]
    print(country)
    for year in data_country.year.unique():
        print(year, data_country[data_country.year == year].suicides_no.sum())

Albania
1987 73
1988 63
1989 68
1992 47
1993 73
1994 50
1995 88
1996 89
1997 170
1998 154
1999 139
2000 54
2001 119
2002 133
2003 124
2004 146
2005 0
2006 0
2007 124
2008 160
2009 0
2010 96
Antigua and Barbuda
1985 0
1986 0
1987 0
1988 0
1989 0
1990 1
1991 0
1992 0
1993 1
1994 0
1995 0
1998 0
1999 0
2000 2
2001 2
2002 0
2003 1
2004 2
2005 0
2006 0
2007 1
2008 0
2009 0
2012 0
2013 0
2014 0
2015 1
Argentina
1985 1988
1986 2284
1987 2286
1988 2354
1989 2247
1990 2140
1991 1951
1992 2184
1993 2230
1994 2241
1995 2204
1996 2229
1997 2184
1998 2347
1999 2423
2000 2774
2001 3132
2002 3162
2003 3289
2004 3121
2005 3058
2006 3108
2007 2946
2008 3059
2009 2884
2010 2943
2011 2912
2012 3248
2013 2987
2014 3231
2015 3073
Armenia
1990 93
1991 80
1992 83
1993 106
1994 112
1995 130
1996 87
1997 77
1998 66
1999 67
2000 61
2001 60
2002 74
2003 59
2006 78
2007 71
2008 63
2009 53
2010 73
2011 67
2012 79
2013 67
2014 58
2015 74
2016 67
Aruba
1995 8
1999 9
2000 10
2001 4
2002 9
2003 2
2004 8
2005 10
2006 9

Iceland
1985 32
1986 37
1987 36
1988 35
1989 22
1990 40
1991 37
1992 28
1993 26
1994 25
1995 27
1996 33
1997 33
1998 30
1999 31
2000 50
2001 36
2002 30
2003 26
2004 35
2005 33
2006 32
2007 37
2008 38
2009 36
2010 46
2011 27
2012 37
2013 49
2014 44
2015 40
2016 40
Ireland
1985 276
1986 282
1987 245
1988 266
1989 278
1990 334
1991 345
1992 362
1993 326
1994 394
1995 403
1996 408
1997 466
1998 498
1999 424
2000 463
2001 488
2002 449
2003 454
2004 464
2005 451
2006 452
2007 458
2008 506
2009 527
2010 486
2011 554
2012 542
2013 487
2014 486
Israel
1985 234
1986 278
1987 271
1988 303
1989 353
1990 304
1991 359
1992 381
1993 371
1994 416
1995 360
1996 306
1997 379
1998 318
1999 368
2000 394
2001 398
2002 382
2003 417
2004 411
2005 425
2006 367
2007 326
2008 354
2009 404
2010 480
2011 409
2012 430
2013 372
2014 382
2015 342
Italy
1985 4759
1986 4691
1987 4516
1988 4381
1989 4342
1990 4402
1991 4453
1992 4543
1993 4697
1994 4536
1995 4569
1996 4689
1997 4694
1998 4504
1999 4115
2000 4108
2001 4

2004 383
2005 458
2006 514
2007 417
2008 441
2009 372
2010 393
2011 358
2012 487
2013 587
2014 580
2015 482
Spain
1985 2514
1986 2755
1987 2804
1988 2960
1989 2987
1990 2939
1991 2916
1992 2786
1993 3037
1994 3171
1995 3157
1996 3320
1997 3373
1998 3261
1999 3220
2000 3394
2001 3189
2002 3372
2003 3478
2004 3508
2005 3399
2006 3247
2007 3264
2008 3457
2009 3429
2010 3158
2011 3180
2012 3539
2013 3873
2014 3911
2015 3604
Sri Lanka
1985 5668
1986 5345
1996 4898
1997 5887
1998 5518
1999 5619
2000 5252
2001 4590
2002 4382
2003 4467
2006 4015
Suriname
1985 80
1986 42
1987 52
1988 49
1989 53
1990 26
1991 57
1992 48
1995 55
1996 21
1997 32
1998 29
1999 49
2000 52
2001 72
2002 69
2003 77
2004 67
2005 72
2006 112
2007 119
2008 147
2009 131
2010 136
2011 127
2012 133
2013 131
2014 128
Sweden
1987 1541
1988 1590
1989 1579
1990 1471
1991 1483
1992 1355
1993 1373
1994 1324
1995 1348
1996 1253
1997 1201
1998 1231
1999 1220
2000 1131
2001 1197
2002 1180
2003 1112
2004 1155
2005 1219
2006 1197
2007 11

### 4.2 Посчитать количество самоубийств по поколениям и возрасту, независимо от страны и года

должены получится две одномерные матрицы и аналог двумерной матрицы. (количество самоубийств по возрастам, поколениям. и по возрастам и поколениям одновременно)

In [61]:
years = data.year.unique()
generations = data.generation.unique()

In [62]:
for year in years:
    print(year, data[data.year == year].suicides_no.sum())

1987 126842
1988 121026
1989 160244
1992 211473
1993 221565
1994 232063
1995 243544
1996 246725
1997 240745
1998 249591
1999 256119
2000 255832
2001 250652
2002 256095
2003 256079
2004 240861
2005 234375
2006 233361
2007 233408
2008 235447
2009 243487
2010 238702
1985 116063
1986 120670
1990 193361
1991 198020
2012 230160
2013 223199
2014 222984
2015 203640
2011 236484
2016 15603


In [63]:
for generation in generations:
    print(generation, data[data.generation == generation].suicides_no.sum())

Generation X 1532804
Silent 1781744
G.I. Generation 510009
Boomers 2284498
Millenials 623459
Generation Z 15906


In [67]:
res = pd.DataFrame(index=years, columns=generations)
for year in years:
    data_year = data[data.year == year]
    for generation in generations:
        res.loc[year, generation] = data_year[data_year.generation == generation].suicides_no.sum()
        print(year, generation, data_year[data_year.generation == generation].suicides_no.sum())

1987 Generation X 17367
1987 Silent 40649
1987 G.I. Generation 46517
1987 Boomers 22309
1987 Millenials 0
1987 Generation Z 0
1988 Generation X 16174
1988 Silent 38770
1988 G.I. Generation 44750
1988 Boomers 21332
1988 Millenials 0
1988 Generation Z 0
1989 Generation X 19786
1989 Silent 53541
1989 G.I. Generation 57490
1989 Boomers 29427
1989 Millenials 0
1989 Generation Z 0
1992 Generation X 24727
1992 Silent 51506
1992 G.I. Generation 20744
1992 Boomers 112809
1992 Millenials 1687
1992 Generation Z 0
1993 Generation X 25262
1993 Silent 54278
1993 G.I. Generation 20522
1993 Boomers 119714
1993 Millenials 1789
1993 Generation Z 0
1994 Generation X 27296
1994 Silent 56576
1994 G.I. Generation 20499
1994 Boomers 125932
1994 Millenials 1760
1994 Generation Z 0
1995 Generation X 71832
1995 Silent 59138
1995 G.I. Generation 20901
1995 Boomers 89752
1995 Millenials 1921
1995 Generation Z 0
1996 Generation X 72958
1996 Silent 59979
1996 G.I. Generation 20643
1996 Boomers 91280
1996 Millenials

In [68]:
res

,Generation X,Silent,G.I. Generation,Boomers,Millenials,Generation Z
1987,17367,40649,46517,22309,0,0
1988,16174,38770,44750,21332,0,0
1989,19786,53541,57490,29427,0,0
1992,24727,51506,20744,112809,1687,0
1993,25262,54278,20522,119714,1789,0
1994,27296,56576,20499,125932,1760,0
1995,71832,59138,20901,89752,1921,0
1996,72958,59979,20643,91280,1865,0
1997,71315,58672,20866,88012,1880,0
1998,72523,61642,21717,91627,2082,0
